# Cold YouTuber

In [10]:
import os 
import numpy as np
import random
import math
from sklearn.metrics import average_precision_score
import pandas as pd
import json

In [11]:
#從grid_search_weight中找尋不同的file 
path = './Result/Result_Matrix/'
files = os.listdir(path)
files

['Cold_YouTuber_y20_35_Edims200.npz',
 'Cold_YouTuber_y20_ACF_NRImg0_Edims200.npz',
 'Cold_YouTuber_y20_ACF_RImg2_Edims200.npz',
 'Cold_YouTuber_y20_VBPR5_Edims200.npz']

In [12]:
def square_sum(pars):
    all_squ_sum = []
    for par in pars:
        all_squ_sum.append(np.sum(np.multiply(par,par)))
    return all_squ_sum 
def shape_sum(pars):
    all_shape = []
    for par in pars:
        all_shape.append(par.shape)
    return all_shape
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))  

In [13]:
num_of_YouTuber = '20'
user_following = np.load('user_following_74_20.npy')
user_category = np.load('user_category_74.npy')
YouTuber_category = np.load('YouTuber_category_0.7_20.npy')
active_users = np.load('active_userID_74.npy')
with open('coldy_ytList.json') as json_file:
    ytList = json.load(json_file)
user_test_amount = 15
yt_test_amount = 10
yt_num = user_following.shape[1]

In [14]:
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
print('user_category after normalized by max...')
print('user_category_norm shape ',user_category_norm.shape)

user_idx = [i for i in range(len(user_following))]
random.seed(5)
more_than_one = []
only_one = []
for i in range(len(user_following)):
    if np.sum(user_following,axis=1)[i] > 1:
        more_than_one.append(i)
    elif np.sum(user_following,axis=1)[i] == 1:
        only_one.append(i)
test_idx = more_than_one
rest_test_idx = random.sample(only_one,user_test_amount-len(more_than_one))
test_idx = sorted(test_idx + rest_test_idx)
print('test_idx',test_idx)
#train and test
train_t = []
train_f = []
test_t = []
test_f = []

for i in range(len(user_following)):
    if i in more_than_one: #如果i = 那九個>1的
        #print(i)
        temp_t = [j for j,v in enumerate(user_following[i]) if v==1]
        temp_f = [j for j,v in enumerate(user_following[i]) if v==0]
        t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
        f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))
        test_t.append(t_for_test)
        test_f.append(f_for_test)
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t.append(t_for_train)
        train_f.append(f_for_train)
    elif i in rest_test_idx: #only for test
        #print(i)
        temp_t = [j for j,v in enumerate(user_following[i]) if v==1]
        t_for_test = temp_t
        temp_f = [j for j,v in enumerate(user_following[i]) if v==0]
        f_for_test = random.sample(temp_f,yt_test_amount-len(t_for_test))
        test_t.append(t_for_test)
        test_f.append(f_for_test)
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t.append(t_for_train)
        train_f.append(f_for_train)
    else:
        t_for_train = [j for j,v in enumerate(user_following[i]) if v==1]
        f_for_train = [j for j,v in enumerate(user_following[i]) if v==0]
        train_t.append(t_for_train)
        train_f.append(f_for_train)
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/user_test_amount
print(avg)

user_category after normalized by max...
user_category_norm shape  (74, 17)
test_idx [16, 23, 24, 35, 36, 39, 40, 45, 50, 52, 53, 58, 61, 63, 69]
1.0666666666666667


In [15]:
test_t

[[2],
 [8],
 [18],
 [15],
 [3],
 [10],
 [2],
 [6, 2],
 [14],
 [7],
 [7],
 [1],
 [14],
 [4],
 [11]]

In [32]:
def getScoreMatrix(RS):
    #取出test的資料
    testRS = np.zeros((user_test_amount,yt_test_amount)) #shape 15*10
    target = np.zeros((user_test_amount,yt_test_amount))
    #test_t 是true的
    #test_f 是false的
    ytMatrix = []
    for z in range(user_test_amount):
        #print(z)
        ytNames = []
        user_id = sorted(test_idx)[z]
        #positive target YouTuber list
        youtube_t = test_t[z] 
        #not target YouTuber list
        youtube_f = test_f[z]
        for yt in youtube_t:
            ytNames.append(ytList[yt])
        for yt in youtube_f:
            ytNames.append(ytList[yt])
        ytMatrix.append(ytNames)
        #前兩個放target的RS
        for i in range(len(youtube_t)):
            testRS[z][i] = RS[z][youtube_t[i]]
            target[z][i] = user_following[user_id][youtube_t[i]]
        for i in range(len(youtube_f)):
            testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]
            target[z][i+len(youtube_t)] = user_following[user_id][youtube_f[i]]
        print('user id, testRS',user_id,testRS[z],youtube_t+youtube_f)
    sumtarget = 0
    for i in range(len(target)):
        #print(np.sum(target[i]))
        sumtarget += np.sum(target[i])
    print('num of positive data in testing:',sumtarget)
    print('total testing data:',user_test_amount*yt_test_amount)
    #print(target)
    #print(testRS)
    return target, testRS,sumtarget,ytMatrix

def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

In [33]:
def getTOP1(target,testRS,sumtarget):
    print('top1')
    correct = 0
    for i in range(len(testRS)):
        top_0 = topN(list(testRS[i]),1) #取一個
        print(top_0)
        #print(top_0)
        if target[i][top_0[0]] == 1:
            correct +=1
        """if top_0[0] < int(np.sum(target[i])):
            correct += 1"""
    top1_prec = correct/len(testRS)
    top1_recall = correct/(sumtarget)
    print('prec ',top1_prec,'recall ',top1_recall)
    #f1 score
    print('F1_score:',F1_score(top1_prec,top1_recall))
    return top1_prec,top1_recall,F1_score(top1_prec,top1_recall)
def getTOP3(target,testRS,sumtarget):
    print('top3')
    correct = 0
    for i in range(len(testRS)):
        top_3 = topN(list(testRS[i]),3) #取一個
        
        #print(top_3)
        for j in range(len(top_3)):
            if target[i][top_3[j]] == 1:
                correct += 1
    top3_prec = correct/(len(testRS)*3)
    top3_recall = correct/(sumtarget)
    print('prec ',top3_prec,'recall ',top3_recall)
    #f1 score
    print('F1_score:',F1_score(top3_prec,top3_recall))
    return top3_prec,top3_recall,F1_score(top3_prec,top3_recall)
def getTOP5(target,testRS,sumtarget):
    print('top5')
    correct = 0
    for i in range(len(testRS)):
        top_5 = topN(list(testRS[i]),5) #取一個
       
        #print(top_5)
        for j in range(len(top_5)):
            if target[i][top_5[j]] == 1:
                correct += 1
    top5_prec = correct/(len(testRS)*5)
    top5_recall = correct/(sumtarget)
    print('prec ',top5_prec,'recall ',top5_recall)
    #f1 score
    print('F1_score:',F1_score(top5_prec,top5_recall))
    return top5_prec,top5_recall,F1_score(top5_prec,top5_recall)

In [34]:
"""
NDCG
https://daiwk.github.io/posts/nlp-ndcg.html
"""
# pre_list
"""
test_amount = 150
yt_test_amount = 18
"""
def NDCG(target,testRS): #target是真正的喜好
    all_sort = []
    num_ndcg = 10
    pre_matrix = np.zeros(shape=(user_test_amount,yt_test_amount)) #(150,18)
    for i in range(user_test_amount): #user amount = 150
        top_n = topN(list(testRS[i]),num_ndcg) #取10個
        #print(top_n)
        all_sort.append(topN(list(testRS[i]),len(testRS[i])))
        #print('all_sort',topN(list(testRS[i]),len(testRS[i])))
        for j in range(len(top_n)):
            pre_matrix[i][top_n[j]] = 1

    #Ideal DCG，理想状况下的DCG。也就是说，相关性完全由高到低排序时算出的DCG：
    def IDCG(ideal_list): #ideal_list example = [1,1,1,1,1,0,0,....]
        idcg=0
        print('ideal',ideal_list)
        for i in range(len(ideal_list)):
            #print((2**true_list[i]-1),math.log2(i+2))
            idcg+= (2**ideal_list[i]-1)/ math.log2(i+2)
        #print('idcg',idcg)
        return idcg
    def DCG(prec_list): #找出前n名的[1,1,1,0,...]
        dcg=0
        print('prec',prec_list)
        for i in range(len(prec_list)):
            dcg+= (2**prec_list[i]-1)/ math.log2(i+2)
        #print('dcg',dcg)
        return dcg
    total_ndcg = 0
    
    for m in range(user_test_amount): # the number of testing users
        print('all_sort',all_sort[m][:num_ndcg])
        idcg = IDCG(target[m][:num_ndcg])
        pre_list = []
        least_pre_list = []
        for s in all_sort[m][:num_ndcg]:
            #print(m,s,target[m][s])
            pre_list.append(target[m][s]) #把prec_list 的 score加進去
        for s in all_sort[m][num_ndcg:]:
            #print(s)
            #print(target[m][s])
            least_pre_list.append(target[m][s]) #把prec_list 的 score加進去
        dcg = DCG(pre_list)
        ndcg = dcg/idcg
        #print(ndcg)
        total_ndcg += ndcg
    avg_ndcg = total_ndcg/user_test_amount
    print('NDCG:',avg_ndcg)
    return pre_matrix,avg_ndcg

# MAP
"""
>>> y_true = np.array([0, 0, 1, 1])
>>> y_scores = np.array([0.1, 0.4, 0.35, 0.8])
>>> average_precision_score(y_true, y_scores)
"""
def MAP(target,testRS):
    #print('target:',target)
    #print('testRS:',testRS)
    total_prec = 0
    for u in range(user_test_amount):
        y_true = target[u]
        y_scores = testRS[u]
        total_prec+=average_precision_score(y_true, y_scores)
    Map_value = total_prec/user_test_amount
    print('MAP',Map_value)
    return Map_value

Our Method

In [35]:
all_3374 = np.load('all_2939D_y20.npy')
def testing(U, Y, A, E,Au, Ay, Aa, Av,B):
    
    result=np.zeros((user_test_amount,yt_num))
    RS=np.zeros((user_test_amount,yt_num))
    #test_idx --> Test 的 index
    sum_alpha = 0
    for s in range(user_test_amount):
        #print(s,test_idx[s])

        yes=[]
        #sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        sample = train_t[test_idx[s]]
        #sample=result_yes_id[now]
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
            alpha_a = np.dot(Au[test_idx[s]][sample[a]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[test_idx[s]][sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]][sample[a]],
                    np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]][sample[a]],np.dot(E,np.expand_dims(all_3374[sample[a]],0).T))
            """
            relu part ...
            """
            alpha[a]=np.sum((relu(alpha_a)))*r
            """
            tanh part ...
            """
            #alpha[a]=np.sum((np.tanh(alpha_a)))*r
            
            
        mul=np.zeros((1,64))
        #print('alpha--------',alpha)
        #print('add alpha------------',np.add(alpha,0.000000001))
        added_alpha = np.add(alpha,0.0000000001)
        norm_alpha = added_alpha/np.sum(added_alpha)
        #print('alpha-----------',alpha)
        #print('norm alpha--------------',norm_alpha)
        sum_alpha += np.sum(alpha)
        for i in range(len(sample)):
            mul+=norm_alpha[i]*A[sample[i]] #attention alpha*Ai part
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    
        
        
        for k in range(yt_num):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
    #print('16,2',RS[0][2])
    return RS

In [36]:
#With Embedding
import os 
all_files = os.listdir(path)
new_all_files = []
for file in all_files:
    if 'y'+num_of_YouTuber in file and 'VBPR' not in file and 'ACF' not in file:
        new_all_files.append(file)
all_files = new_all_files
print(all_files)

['Cold_YouTuber_y20_35_Edims200.npz']


In [37]:
par_files = ['Cold_YouTuber_y'+num_of_YouTuber]
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    csv_path = './Result/csvResult/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            A = par_data['A']
            E = par_data['E']
            #W1 = par_data['W1']
            Wu = par_data['Wu']
            Wy = par_data['Wy']
            Wa = par_data['Wa']
            Wv = par_data['Wv']
            B = par_data['B']
            RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv,B)
            target,testRS,sumtarget,ytMatrix = getScoreMatrix(RS)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,testRS)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('_')[3])
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
            print('--------------------------------------------------------------------------------------------')
    #print(NDCG_List)
    #print(DIM_List)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                   'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                  'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                  'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

['Cold_YouTuber_y20']
Cold_YouTuber_y20_35_Edims200.npz
user id, testRS 16 [ 1.36958768  1.85536209  1.42387201  0.46601382  0.49238204  2.06939288
  0.47264327  2.28554753 -0.05874102  2.00500506] [2, 16, 8, 1, 19, 3, 17, 6, 18, 4]
user id, testRS 23 [ 0.86261954  2.21420777  0.97023422  0.5711019   0.35105224 -0.74232494
 -0.38019082  1.48221692 -1.12867648  1.2472609 ] [8, 19, 3, 9, 0, 13, 18, 7, 4, 2]
user id, testRS 24 [ 0.26123045 -1.53719561 -0.05221112 -4.83629098  1.19944044 -0.40184943
 -1.93771953  0.15125997 -1.41777631 -2.2889437 ] [18, 5, 2, 4, 9, 14, 7, 17, 11, 0]
user id, testRS 35 [ 0.03396643 -0.54604516 -0.75201899 -0.39666861 -0.42535073 -1.20013879
 -0.57809491 -1.21297748 -0.02641791 -0.21902033] [15, 0, 6, 18, 5, 13, 2, 4, 16, 3]
user id, testRS 36 [-1.69626817 -4.06469897 -3.2857396  -2.21101627 -2.79989249 -2.87594484
 -3.40407115 -0.76937589 -4.76492344 -3.27448364] [3, 6, 7, 14, 5, 0, 19, 18, 2, 11]
user id, testRS 39 [-1.73056525  0.83246313  2.46886659 -0.3

In [40]:
df = pd.DataFrame()
for i in range(len(target)):
    df[str(i)+'_Youtuber Names'] = ytMatrix[i]
    df[str(i)+'_following_rel'] = target[i]
    df[str(i)+'_RS'] = testRS[i]
df = df.transpose()
df.to_csv('./user推薦表/Cold_YouTuber/Our_YRM.csv')

VBPR

In [41]:
all_3374 = np.load('image_2048D_y20.npy')
print('visual feature shape:',all_3374.shape)

visual feature shape: (20, 2048)


In [42]:
def testing(U, Y, E,B):
    test_amount = 15
    yt_test_amount = 10
    result=np.zeros((test_amount,20))
    RS=np.zeros((test_amount,20))
    #test_idx --> Test 的 index
    print(test_idx)
    sum_alpha = 0
    for s in range(test_amount):
        mul=np.zeros((1,64))
    
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    
        
        
        for k in range(20):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
            
    #print('sum_alpha',sum_alpha)
    return RS

In [43]:
#With Embedding
import os 
all_files = os.listdir(path)
new_all_files = []
for file in all_files:
    if 'y'+num_of_YouTuber in file and 'VBPR' in file and 'ACF' not in file:
        new_all_files.append(file)
all_files = new_all_files
print(all_files)

['Cold_YouTuber_y20_VBPR5_Edims200.npz']


In [44]:
par_files = ['Cold_YouTuber_y'+num_of_YouTuber+'_VBPR']
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    csv_path = './Result/csvResult/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            E = par_data['E']
            B = par_data['B']
            RS = testing(U, Y, E,B)
            target,testRS,sumtarget,ytMatrix = getScoreMatrix(RS)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,testRS)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('_')[3])
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

['Cold_YouTuber_y20_VBPR']
Cold_YouTuber_y20_VBPR5_Edims200.npz
[16, 23, 24, 35, 36, 39, 40, 45, 50, 52, 53, 58, 61, 63, 69]
user id, testRS 16 [-0.91446334  0.32147968 -0.27301101 -0.34508863 -1.17331295  1.05941149
 -0.69048164  0.68726938 -0.83981158  1.25139831] [2, 16, 8, 1, 19, 3, 17, 6, 18, 4]
user id, testRS 23 [-0.01651129  1.91956448  1.03339277  0.26984616  0.24916689 -0.14316639
  0.22426562  1.22988673  0.09742607  0.78374598] [8, 19, 3, 9, 0, 13, 18, 7, 4, 2]
user id, testRS 24 [ 0.85572333 -1.5119245   0.05427184 -5.11793702  1.94435062 -0.04633455
 -1.48546476  0.92505416 -1.09677027 -2.07872934] [18, 5, 2, 4, 9, 14, 7, 17, 11, 0]
user id, testRS 35 [ 0.3053174  -0.56453093 -0.75354437  0.05010694 -0.28327359 -0.98524474
 -0.31869849 -1.15135475  0.06012871 -0.45222959] [15, 0, 6, 18, 5, 13, 2, 4, 16, 3]
user id, testRS 36 [-1.18396382 -3.13329098 -3.16014444 -2.38701633 -2.26260677 -2.92804853
 -3.27504772 -0.88049536 -4.73798077 -2.78863116] [3, 6, 7, 14, 5, 0, 19, 18

In [45]:
df = pd.DataFrame()
for i in range(len(target)):
    df[str(i)+'_Youtuber Names'] = ytMatrix[i]
    df[str(i)+'_following_rel'] = target[i]
    df[str(i)+'_RS'] = testRS[i]
df = df.transpose()
df.to_csv('./user推薦表/Cold_YouTuber/Our_VBPR.csv')

ACF_R

In [33]:
all_3374 = np.load('image_2048D_y20.npy')
print('visual feature shape:',all_3374.shape)

visual feature shape: (20, 2048)


In [34]:
def testing(U, Y, A, E,Au, Ay, Aa, Av):
    print('U shape',U.shape)
    print('Y shape',Y.shape)
    print('A shape',A.shape)
    print('E shape',E.shape)
    test_amount = 15
    yt_test_amount = 10
    result=np.zeros((test_amount,20))
    RS=np.zeros((test_amount,20))
    #test_idx --> Test 的 index
    
    sum_alpha = 0
    for s in range(test_amount):
        #print(s,test_idx[s])

        yes=[]
        #sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        sample = train_t[test_idx[s]]
        #sample=result_yes_id[now]
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
            alpha_a = np.dot(Au[test_idx[s]][sample[a]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[test_idx[s]][sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]][sample[a]],
                    np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]][sample[a]],np.dot(E,np.expand_dims(all_3374[sample[a]],0).T))
            """
            relu part ...
            """
            alpha[a]=np.sum((relu(alpha_a)))*r
            """
            tanh part ...
            """
            #alpha[a]=np.sum((np.tanh(alpha_a)))*r
            
            
        mul=np.zeros((1,64))
        #print('alpha--------',alpha)
        #print('add alpha------------',np.add(alpha,0.000000001))
        added_alpha = np.add(alpha,0.0000000001)
        norm_alpha = added_alpha/np.sum(added_alpha)
        #print('alpha-----------',alpha)
        #print('norm alpha--------------',norm_alpha)
        sum_alpha += np.sum(alpha)
        for i in range(len(sample)):
            mul+=norm_alpha[i]*A[sample[i]] #attention alpha*Ai part
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    
        
        
        for k in range(20):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)
            
    print('sum_alpha',sum_alpha)
    return RS

In [35]:
import os 
all_files = os.listdir(path)
new_all_files = []
for file in all_files:
    if 'y'+num_of_YouTuber in file and 'VBPR' not in file and 'ACF_R' in file:
        new_all_files.append(file)
all_files = new_all_files
print(all_files)

['Cold_YouTuber_y20_ACF_RImg0_Edims200.npz', 'Cold_YouTuber_y20_ACF_RImg1_Edims200.npz', 'Cold_YouTuber_y20_ACF_RImg2_Edims200.npz', 'Cold_YouTuber_y20_ACF_RImg3_Edims200.npz', 'Cold_YouTuber_y20_ACF_RImg4_Edims200.npz', 'Cold_YouTuber_y20_ACF_RImg5_Edims200.npz', 'Cold_YouTuber_y20_ACF_RImg6_Edims200.npz', 'Cold_YouTuber_y20_ACF_RImg7_Edims200.npz', 'Cold_YouTuber_y20_ACF_RImg8_Edims200.npz', 'Cold_YouTuber_y20_ACF_RImg9_Edims200.npz']


In [36]:
par_files = ['Cold_YouTuber_y'+num_of_YouTuber+'_ACF_R']
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    csv_path = './Result/csvResult/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            A = par_data['A']
            E = par_data['E']
            #W1 = par_data['W1']
            Wu = par_data['Wu']
            Wy = par_data['Wy']
            Wa = par_data['Wa']
            Wv = par_data['Wv']
            RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv)
            target,testRS,sumtarget,ytMatrix = getScoreMatrix(RS)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,testRS)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('_')[4])
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
            print('--------------------------------------------------------------------------------------------')
    #print(NDCG_List)
    #print(DIM_List)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                   'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                  'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                  'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

['Cold_YouTuber_y20_ACF_R']
Cold_YouTuber_y20_ACF_RImg0_Edims200.npz
U shape (74, 64)
Y shape (20, 64)
A shape (20, 64)
E shape (200, 2048)
sum_alpha 0.003405734028662299
user id, testRS 16 [-0.0071339   0.36584852  0.09188704  0.02269927 -0.01534396  0.09169863
 -0.04060364  0.15774022 -0.04109957  0.00222575] [2, 16, 8, 1, 19, 3, 17, 6, 18, 4]
user id, testRS 23 [ 0.04287321  0.04904176 -0.13331335 -0.00351101  0.09890536  0.19729574
 -0.02619562  0.07355    -0.21921198 -0.02573127] [8, 19, 3, 9, 0, 13, 18, 7, 4, 2]
user id, testRS 24 [ 0.15122785  0.02032559  0.09484908 -0.02826781  0.20473957 -0.07706293
  0.02487622  0.10788793  0.04929696  0.08738527] [18, 5, 2, 4, 9, 14, 7, 17, 11, 0]
user id, testRS 35 [-0.05302122 -0.26048504  0.24686407 -0.12296598  0.0674806   0.01342803
 -0.23093507  0.00797633  0.23513378  0.002371  ] [15, 0, 6, 18, 5, 13, 2, 4, 16, 3]
user id, testRS 36 [ 0.0568808  -0.23816469  0.05468488  0.0334078   0.13895722  0.00970907
  0.13908153  0.09823971  0.22

U shape (74, 64)
Y shape (20, 64)
A shape (20, 64)
E shape (200, 2048)
sum_alpha 0.004727759680105135
user id, testRS 16 [-0.03863326  0.38831054  0.09750229  0.00939478 -0.02673659  0.07185269
 -0.04952256  0.12445685 -0.01707528  0.03510593] [2, 16, 8, 1, 19, 3, 17, 6, 18, 4]
user id, testRS 23 [ 0.05063734  0.05874131 -0.17123071  0.00188587  0.08790919  0.18533831
 -0.02440193  0.07891549 -0.23678169 -0.04779864] [8, 19, 3, 9, 0, 13, 18, 7, 4, 2]
user id, testRS 24 [ 0.17791201  0.12317274  0.05971493 -0.03311749  0.15933152 -0.05976612
 -0.02017755  0.09726008  0.05247345  0.05703275] [18, 5, 2, 4, 9, 14, 7, 17, 11, 0]
user id, testRS 35 [-0.02691366 -0.26329552  0.22903838 -0.18324734  0.05298949  0.02189145
 -0.2385019   0.0351965   0.23639363  0.03550406] [15, 0, 6, 18, 5, 13, 2, 4, 16, 3]
user id, testRS 36 [ 0.08182971 -0.19456812  0.0626044   0.03568749  0.14313375 -0.03570937
  0.14008331  0.04667353  0.24516815 -0.11394283] [3, 6, 7, 14, 5, 0, 19, 18, 2, 11]
user id, testR

C:\Users\NTU\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in double_scalars


U shape (74, 64)
Y shape (20, 64)
A shape (20, 64)
E shape (200, 2048)
sum_alpha 0.0013786853523924947
user id, testRS 16 [-0.01163243  0.38169748  0.09079135  0.016472   -0.04283126  0.11377288
 -0.05320937  0.13143857 -0.03881879  0.00916009] [2, 16, 8, 1, 19, 3, 17, 6, 18, 4]
user id, testRS 23 [ 0.02711918  0.09005255 -0.11829232 -0.00918995  0.07919105  0.18620851
  0.01009632  0.08733195 -0.22095531 -0.07392744] [8, 19, 3, 9, 0, 13, 18, 7, 4, 2]
user id, testRS 24 [ 0.10783889  0.08110597  0.1035776  -0.02596492  0.20531079 -0.06227775
  0.01594437  0.05558941  0.04814941  0.09033311] [18, 5, 2, 4, 9, 14, 7, 17, 11, 0]
user id, testRS 35 [-0.02897233 -0.24053774  0.23659508 -0.14727065  0.0306021   0.03172447
 -0.23870692  0.0082432   0.22615889  0.07568788] [15, 0, 6, 18, 5, 13, 2, 4, 16, 3]
user id, testRS 36 [ 0.05107878 -0.19536115  0.06721567  0.02120764  0.12593633  0.02543754
  0.13872128  0.05995205  0.24995502 -0.09791847] [3, 6, 7, 14, 5, 0, 19, 18, 2, 11]
user id, test

U shape (74, 64)
Y shape (20, 64)
A shape (20, 64)
E shape (200, 2048)
sum_alpha 0.0014620096874697244
user id, testRS 16 [-0.00866175  0.35206611  0.09722442  0.02710885 -0.05039391  0.07967434
 -0.0262527   0.14414591 -0.02844598  0.01079351] [2, 16, 8, 1, 19, 3, 17, 6, 18, 4]
user id, testRS 23 [ 0.03250552  0.0886804  -0.12872454  0.01435347  0.0887922   0.1796232
 -0.01388996  0.1196481  -0.24938422 -0.0753263 ] [8, 19, 3, 9, 0, 13, 18, 7, 4, 2]
user id, testRS 24 [ 0.14716484  0.01606057  0.11544734 -0.00256348  0.15605086 -0.07350867
  0.00503528  0.0991472   0.05614204  0.05852201] [18, 5, 2, 4, 9, 14, 7, 17, 11, 0]
user id, testRS 35 [-0.03118163 -0.24933739  0.23308937 -0.12725145  0.07829269  0.02833593
 -0.27026795  0.00992107  0.2072633   0.05158388] [15, 0, 6, 18, 5, 13, 2, 4, 16, 3]
user id, testRS 36 [ 0.06454165 -0.20222781  0.04590643  0.00862831  0.13975865 -0.03837117
  0.16746523  0.09936691  0.26780455 -0.09011843] [3, 6, 7, 14, 5, 0, 19, 18, 2, 11]
user id, testR

MAP 0.1979365079365079
--------------------------------------------------------------------------------------------


ACF_NR

In [46]:
def testing(U, Y, A, E,Au, Ay, Aa, Av):

    test_amount = 15
    yt_test_amount = 10
    result=np.zeros((test_amount,20))
    RS=np.zeros((test_amount,20))
    #test_idx --> Test 的 index
    sum_alpha = 0
    for s in range(test_amount):
        #print(s,test_idx[s])

        yes=[]
        #sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        sample = train_t[test_idx[s]]
        #sample=result_yes_id[now]
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            alpha_a = np.dot(Au[test_idx[s]][sample[a]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[test_idx[s]][sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]][sample[a]],
                    np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]][sample[a]],np.dot(E,np.expand_dims(all_3374[sample[a]],0).T))
            """
            relu part ...
            """
            alpha[a]=np.sum((relu(alpha_a)))
            """
            tanh part ...
            """
            #alpha[a]=np.sum((np.tanh(alpha_a)))*r
            
            
        mul=np.zeros((1,64))
        #print('alpha--------',alpha)
        #print('add alpha------------',np.add(alpha,0.000000001))
        added_alpha = np.add(alpha,0.0000000001)
        norm_alpha = added_alpha/np.sum(added_alpha)
        #print('alpha-----------',alpha)
        #print('norm alpha--------------',norm_alpha)
        sum_alpha += np.sum(alpha)
        for i in range(len(sample)):
            mul+=norm_alpha[i]*A[sample[i]] #attention alpha*Ai part
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    
        
        
        for k in range(20):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)
            
    print('sum_alpha',sum_alpha)
    return RS

In [47]:
import os 
all_files = os.listdir(path)
new_all_files = []
for file in all_files:
    if 'y'+num_of_YouTuber in file and 'VBPR' not in file and 'ACF_NR' in file:
        new_all_files.append(file)
all_files = new_all_files
print(all_files)

['Cold_YouTuber_y20_ACF_NRImg0_Edims200.npz']


In [48]:
par_files = ['Cold_YouTuber_y'+num_of_YouTuber+'_ACF_NR']
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    csv_path = './Result/csvResult/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            A = par_data['A']
            E = par_data['E']
            #W1 = par_data['W1']
            Wu = par_data['Wu']
            Wy = par_data['Wy']
            Wa = par_data['Wa']
            Wv = par_data['Wv']
            RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv)
            target,testRS,sumtarget,ytMatrix = getScoreMatrix(RS)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,testRS)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('_')[4])
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
            print('--------------------------------------------------------------------------------------------')
    #print(NDCG_List)
    #print(DIM_List)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                   'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                  'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                  'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

['Cold_YouTuber_y20_ACF_NR']
Cold_YouTuber_y20_ACF_NRImg0_Edims200.npz
sum_alpha 0.001232840440934524
user id, testRS 16 [-0.01797464  0.31613185  0.12740205  0.00539005  0.03182097  0.06383312
 -0.08097266  0.11165281 -0.02344608 -0.04339881] [2, 16, 8, 1, 19, 3, 17, 6, 18, 4]
user id, testRS 23 [ 0.0557389   0.04231967 -0.04407557 -0.02610449  0.04406761  0.1773058
  0.06534317  0.13916048 -0.26813459 -0.03472526] [8, 19, 3, 9, 0, 13, 18, 7, 4, 2]
user id, testRS 24 [ 0.15745042  0.07509531  0.08159123  0.01185288  0.14980317 -0.08869685
  0.0277376   0.09891884  0.05104561  0.07758308] [18, 5, 2, 4, 9, 14, 7, 17, 11, 0]
user id, testRS 35 [-0.01718268 -0.27735601  0.24035856 -0.14319741  0.03519522  0.01616308
 -0.25826902  0.0162618   0.22565434  0.04232779] [15, 0, 6, 18, 5, 13, 2, 4, 16, 3]
user id, testRS 36 [ 0.1093857  -0.17232059  0.01203076  0.01113867  0.10530786 -0.00664382
  0.11199488  0.09016469  0.22108659 -0.06562672] [3, 6, 7, 14, 5, 0, 19, 18, 2, 11]
user id, testRS

In [49]:
df = pd.DataFrame()
for i in range(len(target)):
    df[str(i)+'_Youtuber Names'] = ytMatrix[i]
    df[str(i)+'_following_rel'] = target[i]
    df[str(i)+'_RS'] = testRS[i]
df = df.transpose()
df.to_csv('./user推薦表/Cold_YouTuber/ACF.csv')

# Cold Users

Data Prepare

In [28]:
import os 
import numpy as np
import random
import math
from sklearn.metrics import average_precision_score
import pandas as pd

In [49]:
#從grid_search_weight中找尋不同的file 
path = './Result/Cold_User/'
files = os.listdir(path)
files

['Cold_User_y57_0_Edims200.npz',
 'Cold_User_y57_10_Edims200.npz',
 'Cold_User_y57_11_Edims200.npz',
 'Cold_User_y57_12_Edims200.npz',
 'Cold_User_y57_13_Edims200.npz',
 'Cold_User_y57_14_Edims200.npz',
 'Cold_User_y57_15_Edims200.npz',
 'Cold_User_y57_16_Edims200.npz',
 'Cold_User_y57_17_Edims200.npz',
 'Cold_User_y57_18_Edims200.npz',
 'Cold_User_y57_19_Edims200.npz',
 'Cold_User_y57_1_Edims200.npz',
 'Cold_User_y57_20_Edims200.npz',
 'Cold_User_y57_2_Edims200.npz',
 'Cold_User_y57_3_Edims200.npz',
 'Cold_User_y57_4_Edims200.npz',
 'Cold_User_y57_5_Edims200.npz',
 'Cold_User_y57_6_Edims200.npz',
 'Cold_User_y57_7_Edims200.npz',
 'Cold_User_y57_8_Edims200.npz',
 'Cold_User_y57_9_Edims200.npz',
 'Cold_User_y57_ACF_NRImg0_Edims200.npz',
 'Cold_User_y57_ACF_NRImg1_Edims200.npz',
 'Cold_User_y57_ACF_RImg0_Edims200.npz',
 'Cold_User_y57_ACF_RImg1_Edims200.npz',
 'Cold_User_y57_VBPR0_Edims200.npz',
 'Cold_User_y57_VBPR1_Edims200.npz',
 'old']

In [50]:
def square_sum(pars):
    all_squ_sum = []
    for par in pars:
        all_squ_sum.append(np.sum(np.multiply(par,par)))
    return all_squ_sum 
def shape_sum(pars):
    all_shape = []
    for par in pars:
        all_shape.append(par.shape)
    return all_shape
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))  

In [51]:
num_of_YouTuber = '57'
user_following = np.load('user_following_127_'+num_of_YouTuber+'.npy')
#all_3374 = np.load('all_2939D_y'+num_of_YouTuber+'.npy')
user_category = np.load('user_category_127.npy')
YouTuber_category = np.load('YouTuber_category_0.7_'+num_of_YouTuber+'.npy')
active_users = np.load('active_userID_127.npy')

user_test_amount = 25
yt_test_amount = 29
yt_num = user_following.shape[1]

In [52]:
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
print('user_category after normalized by max...')
print('user_category_norm shape ',user_category_norm.shape)
print('user cateogory norm',user_category_norm)
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
#print(following_true)
#最少跟最多的following 
minlen = 10000
maxlen = 0
num_of_follower = []
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
    num_of_follower.append(len(following_true[i]))
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)
user_idx = [i for i in range(len(user_following))]
#user_idx = user_idx_over10
#test_idx is the number of user for testing
random.seed(5)
test_idx = sorted(random.sample(user_idx,user_test_amount))
print(test_idx)
# Training  and Testing --New
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*user_test_amount
test_f = [0]*user_test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(user_following.shape[1]):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(user_following.shape[1]):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        #print(len(temp_t),math.ceil(0.5*len(temp_t)))
        t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
        f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))
        
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        #print(len(t_for_train ))
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/user_test_amount
print(avg)

user_category after normalized by max...
user_category_norm shape  (127, 17)
user cateogory norm [[0.17808219 0.         0.         ... 0.         0.         0.        ]
 [0.00884956 0.0159292  0.00176991 ... 0.         0.         0.00176991]
 [0.         0.1        0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.03448276 0.         0.         ... 0.         0.         0.03448276]
 [0.03846154 0.         0.         ... 0.         0.         0.        ]]
Min number of followings  5
Max number of followings  5
[3, 6, 14, 20, 31, 32, 45, 47, 48, 59, 60, 67, 69, 79, 83, 88, 94, 99, 101, 107, 111, 117, 119, 120, 124]
3.0


Our Method

In [53]:
def testing(U, Y, A, E,Au, Ay, Aa, Av,B):
    
    result=np.zeros((user_test_amount,yt_num))
    RS=np.zeros((user_test_amount,yt_num))
    #test_idx --> Test 的 index
    sum_alpha = 0
    for s in range(user_test_amount):
        #print(s,test_idx[s])

        yes=[]
        #sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        sample = train_t[test_idx[s]]
        #sample=result_yes_id[now]
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
            alpha_a = np.dot(Au[test_idx[s]][sample[a]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[test_idx[s]][sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]][sample[a]],
                    np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]][sample[a]],np.dot(E,np.expand_dims(all_3374[sample[a]],0).T))
            """
            relu part ...
            """
            alpha[a]=np.sum((relu(alpha_a)))*r
            """
            tanh part ...
            """
            #alpha[a]=np.sum((np.tanh(alpha_a)))*r
            
            
        mul=np.zeros((1,64))
        #print('alpha--------',alpha)
        #print('add alpha------------',np.add(alpha,0.000000001))
        added_alpha = np.add(alpha,0.0000000001)
        norm_alpha = added_alpha/np.sum(added_alpha)
        #print('alpha-----------',alpha)
        #print('norm alpha--------------',norm_alpha)
        sum_alpha += np.sum(alpha)
        for i in range(len(sample)):
            mul+=norm_alpha[i]*A[sample[i]] #attention alpha*Ai part
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    
        
        
        for k in range(yt_num):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
            
    return RS

In [54]:
#With Embedding
import os 
all_files = os.listdir(path)
new_all_files = []
for file in all_files:
    if 'y'+num_of_YouTuber in file and 'VBPR' not in file and 'ACF' not in file:
        new_all_files.append(file)
all_files = new_all_files
print(all_files)

['Cold_User_y57_0_Edims200.npz', 'Cold_User_y57_10_Edims200.npz', 'Cold_User_y57_11_Edims200.npz', 'Cold_User_y57_12_Edims200.npz', 'Cold_User_y57_13_Edims200.npz', 'Cold_User_y57_14_Edims200.npz', 'Cold_User_y57_15_Edims200.npz', 'Cold_User_y57_16_Edims200.npz', 'Cold_User_y57_17_Edims200.npz', 'Cold_User_y57_18_Edims200.npz', 'Cold_User_y57_19_Edims200.npz', 'Cold_User_y57_1_Edims200.npz', 'Cold_User_y57_20_Edims200.npz', 'Cold_User_y57_2_Edims200.npz', 'Cold_User_y57_3_Edims200.npz', 'Cold_User_y57_4_Edims200.npz', 'Cold_User_y57_5_Edims200.npz', 'Cold_User_y57_6_Edims200.npz', 'Cold_User_y57_7_Edims200.npz', 'Cold_User_y57_8_Edims200.npz', 'Cold_User_y57_9_Edims200.npz']


In [56]:
par_files = ['Cold_User_y'+num_of_YouTuber]
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    csv_path = './Result/csvResult/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            A = par_data['A']
            E = par_data['E']
            #W1 = par_data['W1']
            Wu = par_data['Wu']
            Wy = par_data['Wy']
            Wa = par_data['Wa']
            Wv = par_data['Wv']
            B = par_data['B']
            RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv,B)
            target,testRS,sumtarget = getScoreMatrix(RS)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,testRS)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('_')[3])
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
            print('--------------------------------------------------------------------------------------------')
    #print(NDCG_List)
    #print(DIM_List)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                   'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                  'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                  'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

['Cold_User_y57']
Cold_User_y57_0_Edims200.npz
user id, testRS 3 [-0.99854234  0.57086949 -1.46446051 -1.15736038 -1.8118501   0.80199415
  0.53571416 -1.26930819 -0.48123706 -0.75926097 -1.72323267 -2.60465203
 -1.93875423 -1.96866135 -1.76753987 -1.36731041  0.49419853 -2.03894667
 -0.8036458  -3.10476822 -2.7286847  -3.76598379 -2.43484872 -3.01106315
 -0.20914642  1.30939765  0.01790926 -1.84153139 -1.83372996] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [ 0.47827703  1.55719641  1.34973061  0.24603676 -1.07387672 -5.03560079
 -1.32597452  4.05467849 -1.20314354 -1.51472527 -1.85351933 -1.8437165
 -1.57706862  0.5307836  -6.21241356  1.56979756 -2.2516814  -3.12703218
 -1.17087796  0.1197314  -4.10319922  1.33002824 -1.81203799  0.33150642
 -4.7707176   3.32686681  0.36540533 -3.76631163 -1.27912474] [10, 51, 17, 14, 27, 22, 1, 26, 29, 12, 11, 19, 4, 24, 54, 41, 40, 0, 44, 45, 43, 25, 37, 33, 36,

user id, testRS 3 [-1.97882348  0.75433426 -0.58463354 -1.88758169 -3.45025553 -0.63564662
  0.04035397 -1.37682666 -1.0674118  -2.0532865  -3.9116806  -2.66994615
 -3.07214326 -1.72481642 -3.19159852 -2.52927505  0.12602717 -2.34330209
 -1.71563517 -2.11437174 -2.57985112 -3.79857793 -1.80347957 -3.18542796
 -1.64551729 -0.31117303 -0.5222488  -2.21263512 -2.87843399] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [-1.45610323  0.72167714  0.83644325 -2.9927537  -2.13229319 -5.3572707
 -2.65616557  2.55088517 -0.11821992 -3.30804009 -1.97404939 -2.3353574
 -3.23150595 -0.83025717 -5.61748971  0.45391804 -3.58284136 -3.65741282
 -2.15795713 -0.54566191 -4.15272231  1.35410533 -3.0591209  -0.29107582
 -5.35554333  2.1573853  -0.71785407 -4.59880877 -2.43392881] [10, 51, 17, 14, 27, 22, 1, 26, 29, 12, 11, 19, 4, 24, 54, 41, 40, 0, 44, 45, 43, 25, 37, 33, 36, 49, 5, 18, 30]
user id, testRS 14 [ 4.13215778 

user id, testRS 3 [-2.25307209 -0.26038669 -1.79426897 -2.45777204 -4.30369183 -1.76369659
 -0.71243919 -1.44591319 -0.74991789 -2.98457883 -3.57815608 -3.46547833
 -3.26984797 -2.54134885 -3.68883502 -2.95196946 -0.59953047 -3.24441841
 -2.42935456 -2.94083009 -3.19670481 -4.89606129 -2.37240576 -4.37376867
 -2.53942266 -1.38484155 -0.87941581 -2.37220091 -3.93210648] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [-1.49615131  1.42372719  0.41419916 -3.12805567 -0.76367822 -5.3522974
 -3.44414529  1.58455151 -1.9860441  -3.28797484 -2.46545934 -2.69454269
 -1.60764943  0.21559856 -6.04326414  0.78589405 -3.20443269 -4.48476627
 -1.93247996 -0.72498135 -4.20088856  0.86492502 -2.16636978 -1.06595062
 -5.22453373  1.42530122 -1.9203512  -5.31488938 -2.23873367] [10, 51, 17, 14, 27, 22, 1, 26, 29, 12, 11, 19, 4, 24, 54, 41, 40, 0, 44, 45, 43, 25, 37, 33, 36, 49, 5, 18, 30]
user id, testRS 14 [ 3.44567641

user id, testRS 3 [-2.05018604 -0.24839499 -1.57161252 -3.09018765 -2.31618717 -1.54294575
 -0.55619098 -1.84763471 -1.25725235 -2.32416602 -3.87843374 -3.00265669
 -3.0960016  -1.99082618 -2.9900817  -2.63316152 -0.42320837 -2.74817263
 -1.34823087 -2.58301113 -2.8662414  -4.47504392 -3.116916   -3.73316886
 -2.09845429 -1.0901299  -1.2824023  -2.99680987 -3.64725605] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [-1.32986876  0.45583322  0.59718452 -2.22047343 -1.91251846 -4.99086122
 -2.53103748  1.18430234 -0.52599834 -2.07508254 -2.09875887 -2.57570234
 -3.20844571 -0.09397373 -5.8698088   0.63621668 -3.28600636 -4.14070082
 -2.06728391 -0.57398537 -3.86582284 -0.072616   -2.68987942 -0.96942093
 -5.08307937  2.34576952 -1.28898538 -4.95856719 -2.15507535] [10, 51, 17, 14, 27, 22, 1, 26, 29, 12, 11, 19, 4, 24, 54, 41, 40, 0, 44, 45, 43, 25, 37, 33, 36, 49, 5, 18, 30]
user id, testRS 14 [ 3.1875647

user id, testRS 3 [-0.84594094  1.36330826 -2.11496757  0.02134967 -2.31579081  0.52211101
  0.49635946 -2.39672353 -0.32636919 -0.51379188 -0.89909222 -3.46899872
 -1.05319568 -2.76478194 -0.89728205 -0.34151907  1.4471009  -2.78244576
 -1.76770589 -3.95015447 -3.19039112 -5.16678972 -2.19834829 -4.005603
  0.47152717  0.59764012  1.5903717   0.78974876 -1.23086976] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [ 0.87244874  3.41652097  0.99080549 -1.53588442  2.06633378 -5.5695245
 -2.63083057  2.49030948 -2.82198744 -2.02123241 -3.38288144 -1.96557625
  1.06189346  0.45268732 -7.11127729  1.70808331 -0.74824423 -4.31494696
  0.6358665   0.4924035  -4.76788353  1.41909821  1.54505578  1.73040254
 -5.33712041  1.90249877 -2.42623409 -5.57294032 -1.0227783 ] [10, 51, 17, 14, 27, 22, 1, 26, 29, 12, 11, 19, 4, 24, 54, 41, 40, 0, 44, 45, 43, 25, 37, 33, 36, 49, 5, 18, 30]
user id, testRS 14 [ 4.00613469  

...

user id, testRS 3 [-0.18702991  1.76180654 -1.21658516  0.42587309 -0.6166418   1.12705864
  1.58526287 -0.87591171  0.8758956   0.72607791 -0.12272334 -2.03321647
 -0.4170447  -1.5526064  -0.31823768  0.22241595  1.18414935 -1.47629828
  0.26037663 -3.24940224 -2.09265384 -3.23154322 -0.38423003 -2.53106102
  0.62321725  2.13896486  1.57256947  1.05976594 -0.7277283 ] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [ 2.66435079  4.83222381  2.98229187  0.81063579  4.09232167 -4.79425263
 -1.0193578   4.78158637 -2.02317846 -0.26231752 -2.21797354 -0.62949203
  3.24086992  1.6381851  -6.67678641  3.03802853  0.90868622 -3.09511331
  2.38715168  2.2533788  -3.42104157  1.03448075  3.25593899  3.16743057
 -4.33826365  3.74115797 -0.40877342 -3.798869    0.34514562] [10, 51, 17, 14, 27, 22, 1, 26, 29, 12, 11, 19, 4, 24, 54, 41, 40, 0, 44, 45, 43, 25, 37, 33, 36, 49, 5, 18, 30]
user id, testRS 14 [ 4.7111341

user id, testRS 3 [-1.35775369  1.04608207 -0.30142639 -1.91789204 -2.06109845  0.18455686
  0.6546335  -1.06540214 -0.10408045 -2.15469956 -3.41048527 -2.44500964
 -2.76190925 -2.11977539 -2.77780405 -1.81931554  0.13435914 -2.35024121
 -0.62831203 -2.95654828 -2.59497478 -3.74096787 -1.0887182  -3.02277561
 -1.37153212  0.61921447 -0.17710466 -1.6945165  -2.50960657] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [ 0.72079204  4.24807815  3.46977616  0.01334316  1.03887277 -4.22513894
 -1.2266545   5.27173113 -1.23404478 -0.40493969 -1.2870306  -0.41123292
 -0.22742713  2.34635167 -5.46283834  3.42635132 -0.45340433 -1.77759849
  0.74278179  3.01688552 -2.95510458  1.3845947   0.15749668  2.25833099
 -3.99870576  3.68686629  0.45800408 -2.36862153  0.9300071 ] [10, 51, 17, 14, 27, 22, 1, 26, 29, 12, 11, 19, 4, 24, 54, 41, 40, 0, 44, 45, 43, 25, 37, 33, 36, 49, 5, 18, 30]
user id, testRS 14 [ 3.2688605

user id, testRS 3 [-3.27266501 -0.36591097 -2.02660659 -3.14372401 -4.24067157 -1.76383816
 -1.019131   -2.28022588 -2.27984378 -4.10787031 -5.32761957 -4.08917042
 -4.4779929  -3.7999186  -4.64866419 -3.64116665 -1.09998195 -3.75191405
 -1.96970977 -4.42170101 -4.26586934 -5.19098942 -2.72891713 -4.70427762
 -2.89126181 -1.49717033 -2.12297937 -4.27222979 -4.16445844] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [-2.56873486  0.03351777  0.24280588 -1.78169814 -2.73949156 -6.02992403
 -2.38011479  0.86062839 -0.74114595 -2.53217572 -2.1727572  -2.66479423
 -3.96523933 -0.82556393 -6.99426687  0.01510113 -3.97513015 -4.38720322
 -2.64832656 -1.16630074 -5.01736584 -0.12833128 -3.39029948 -1.31845622
 -5.61031417  2.20256731 -0.30624225 -5.47255434 -2.89607456] [10, 51, 17, 14, 27, 22, 1, 26, 29, 12, 11, 19, 4, 24, 54, 41, 40, 0, 44, 45, 43, 25, 37, 33, 36, 49, 5, 18, 30]
user id, testRS 14 [ 3.3537821

user id, testRS 3 [-1.36283517  1.61800448 -1.15154898 -1.56283696 -4.05866339  0.42942918
  0.65772988 -1.76438709 -1.09986597 -3.08972402 -3.6210132  -3.62987437
 -2.89221879 -2.35878447 -3.05825118 -1.82427146  0.66217095 -2.92823054
 -2.05997152 -3.973184   -3.32376155 -4.87399051 -3.82001375 -4.37083241
 -0.14990279  0.40343655  0.01999397 -2.55035785 -2.83316637] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [-2.96053195  1.9250727   2.16288256 -3.33675519 -1.24128171 -5.93808576
 -4.16261193  3.33585653 -0.46813839 -2.32036297 -2.42001476 -2.2321452
 -2.45432053 -0.58031035 -5.93803981  2.05957516 -2.64580425 -3.60832542
 -0.71063819  0.95018823 -3.1648031   1.8900394  -2.20359254  2.38745149
 -4.84987134  1.38661612 -1.77324769 -5.13504497 -1.03225407] [10, 51, 17, 14, 27, 22, 1, 26, 29, 12, 11, 19, 4, 24, 54, 41, 40, 0, 44, 45, 43, 25, 37, 33, 36, 49, 5, 18, 30]
user id, testRS 14 [ 3.68251414

user id, testRS 3 [-2.05850197  0.58078592 -1.64868644 -2.09778484 -2.39128162 -1.62167513
 -0.2521773  -1.9407375  -1.49037448 -2.36316137 -3.76862701 -2.87078792
 -2.58663789 -2.24005681 -2.71490068 -2.35528881 -0.03877994 -2.51635778
 -0.92860143 -3.2708974  -2.71544726 -3.81585395 -1.62979855 -3.67724142
 -1.76880448 -1.36537214 -0.63247689 -2.10065146 -3.22811803] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [-1.73326532  1.22677709  0.60896437 -2.28297705 -1.18042087 -5.2659085
 -2.92626565  0.03350746 -1.95337877 -1.92182092 -2.25927419 -2.26129983
 -1.87785381 -0.42358781 -5.65578338 -0.15390482 -2.16237321 -3.45105798
 -1.59311775 -0.24664316 -3.4186738  -0.4000241  -1.03878424 -0.62040857
 -4.39963213  1.66623891 -2.31730562 -4.67707319 -2.00763928] [10, 51, 17, 14, 27, 22, 1, 26, 29, 12, 11, 19, 4, 24, 54, 41, 40, 0, 44, 45, 43, 25, 37, 33, 36, 49, 5, 18, 30]
user id, testRS 14 [ 2.21157745

VBPR

In [58]:
all_3374 = np.load('image_2048D_y57.npy')
print('visual feature shape:',all_3374.shape)

visual feature shape: (57, 2048)


In [68]:
def testing(U, Y, E,B):
    test_amount = 25
    yt_test_amount = 29
    result=np.zeros((test_amount,57))
    RS=np.zeros((test_amount,57))
    #test_idx --> Test 的 index
    print(test_idx)
    sum_alpha = 0
    for s in range(test_amount):
        mul=np.zeros((1,64))
    
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    
        
        
        for k in range(57):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
            
    #print('sum_alpha',sum_alpha)
    return RS

In [69]:
#With Embedding
import os 
all_files = os.listdir(path)
new_all_files = []
for file in all_files:
    if 'y'+num_of_YouTuber in file and 'VBPR' in file and 'ACF' not in file:
        new_all_files.append(file)
all_files = new_all_files
print(all_files)

['Cold_User_y57_VBPR0_Edims200.npz', 'Cold_User_y57_VBPR1_Edims200.npz']


In [70]:
par_files = ['Cold_User_y'+num_of_YouTuber+'_VBPR']
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    csv_path = './Result/csvResult/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            E = par_data['E']
            B = par_data['B']
            RS = testing(U, Y, E,B)
            target,testRS,sumtarget = getScoreMatrix(RS)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,testRS)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('_')[3])
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

['Cold_User_y57_VBPR']
Cold_User_y57_VBPR0_Edims200.npz
[3, 6, 14, 20, 31, 32, 45, 47, 48, 59, 60, 67, 69, 79, 83, 88, 94, 99, 101, 107, 111, 117, 119, 120, 124]
user id, testRS 3 [ 0.15921969  2.12023488 -0.45067717 -0.17440613 -0.25356426  0.47411483
  1.47238502  0.61166275  0.55927807 -0.78595822 -1.30125495 -0.66496943
 -0.5058971  -0.44404776 -0.46315781 -0.81436573  1.48063716 -1.54726886
  0.29823389 -0.92898738 -1.70686894 -2.65362409  0.48563681 -1.56663975
  0.45911941  0.9565288   1.00786659  0.08935685 -0.98045151] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [-0.00335827  2.89377592  2.53040463 -1.59412508 -0.7093076  -0.83759648
 -0.23015059  2.81253155  1.51901102  0.76675135  0.27848392  0.60067288
  0.11857096  1.05388446 -1.96509962  2.22693387  0.71794019 -2.21989126
  0.3696162   2.4075433  -1.30796118  1.44466888  0.56118773  2.52704391
 -2.73233925  1.02547419 -1.60671863 -2.192

user id, testRS 60 [ 2.30714709  2.50236766  2.23287573  0.40048466  1.13137052 -1.24806771
 -0.47228689  0.3465823   0.5340017  -0.65815019 -0.36574298  0.58724274
 -1.40964268 -1.6838857  -0.30518393  0.77309495 -1.72864601 -2.56833402
 -2.19582239 -2.32538776  0.75907001  0.85895929  1.5386112  -0.06559068
  0.32811434  1.66344606  0.11203717  0.02890661  0.32168254] [45, 42, 47, 27, 37, 43, 35, 24, 30, 9, 10, 39, 25, 55, 31, 12, 8, 5, 23, 0, 44, 6, 21, 49, 48, 33, 29, 19, 40]
user id, testRS 67 [ 3.54714049  6.3974737   4.4221279  -1.10689147  1.2264577   3.3977852
  3.51546975  3.49898281  2.10357251  0.48109724 -1.77096755 -0.19276991
  0.32736488  4.46703545 -1.10138561  0.66034313 -1.62651818  0.43552333
 -0.95681832 -0.29385542  1.08588671 -3.6715192   1.27202924 -1.99298558
 -0.07084477  3.57546839  1.58918916  4.95091683 -0.02376114] [35, 37, 10, 34, 19, 33, 50, 26, 52, 11, 43, 49, 17, 20, 56, 47, 29, 1, 22, 46, 45, 55, 9, 54, 0, 3, 23, 40, 15]
user id, testRS 69 [ 0.8918685

ACF_R

In [74]:
all_3374 = np.load('image_2048D_y57.npy')
print('visual feature shape:',all_3374.shape)

visual feature shape: (57, 2048)


In [81]:
def testing(U, Y, A, E,Au, Ay, Aa, Av):
    print('U shape',U.shape)
    print('Y shape',Y.shape)
    print('A shape',A.shape)
    print('E shape',E.shape)
    test_amount = 25
    yt_test_amount = 29
    result=np.zeros((test_amount,57))
    RS=np.zeros((test_amount,57))
    #test_idx --> Test 的 index
    
    sum_alpha = 0
    for s in range(test_amount):
        #print(s,test_idx[s])

        yes=[]
        #sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        sample = train_t[test_idx[s]]
        #sample=result_yes_id[now]
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
            alpha_a = np.dot(Au[test_idx[s]][sample[a]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[test_idx[s]][sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]][sample[a]],
                    np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]][sample[a]],np.dot(E,np.expand_dims(all_3374[sample[a]],0).T))
            """
            relu part ...
            """
            alpha[a]=np.sum((relu(alpha_a)))*r
            """
            tanh part ...
            """
            #alpha[a]=np.sum((np.tanh(alpha_a)))*r
            
            
        mul=np.zeros((1,64))
        #print('alpha--------',alpha)
        #print('add alpha------------',np.add(alpha,0.000000001))
        added_alpha = np.add(alpha,0.0000000001)
        norm_alpha = added_alpha/np.sum(added_alpha)
        #print('alpha-----------',alpha)
        #print('norm alpha--------------',norm_alpha)
        sum_alpha += np.sum(alpha)
        for i in range(len(sample)):
            mul+=norm_alpha[i]*A[sample[i]] #attention alpha*Ai part
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    
        
        
        for k in range(57):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)
            
    print('sum_alpha',sum_alpha)
    return RS

In [82]:
import os 
all_files = os.listdir(path)
new_all_files = []
for file in all_files:
    if 'y'+num_of_YouTuber in file and 'VBPR' not in file and 'ACF_R' in file:
        new_all_files.append(file)
all_files = new_all_files
print(all_files)

['Cold_User_y57_ACF_RImg0_Edims200.npz', 'Cold_User_y57_ACF_RImg1_Edims200.npz']


In [83]:
par_files = ['Cold_User_y'+num_of_YouTuber+'_ACF_R']
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    csv_path = './Result/csvResult/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            A = par_data['A']
            E = par_data['E']
            #W1 = par_data['W1']
            Wu = par_data['Wu']
            Wy = par_data['Wy']
            Wa = par_data['Wa']
            Wv = par_data['Wv']
            RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv)
            target,testRS,sumtarget = getScoreMatrix(RS)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,testRS)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('_')[3])
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
            print('--------------------------------------------------------------------------------------------')
    #print(NDCG_List)
    #print(DIM_List)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                   'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                  'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                  'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

['Cold_User_y57_ACF_R']
Cold_User_y57_ACF_RImg0_Edims200.npz
U shape (127, 64)
Y shape (57, 64)
A shape (57, 64)
E shape (200, 2048)
sum_alpha 0.013644526332883372
user id, testRS 3 [-0.10244212  0.15686723  0.03842921 -0.07117622 -0.1590316   0.33186506
  0.06055607 -0.11379545 -0.12637253  0.17256216 -0.03469568 -0.15729286
  0.00234192 -0.18440007  0.00550625  0.11925583  0.2418557  -0.23161869
 -0.14406798 -0.07245958 -0.0674864  -0.31200149 -0.10642044 -0.20807184
 -0.09381041  0.13221871  0.48457428  0.08197038 -0.09596314] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [ 0.170522    0.63582999  0.18311728 -0.20890616 -0.01144333 -0.27995077
 -0.17759509  0.57028214 -0.19433086 -0.01080854 -0.13823049 -0.34031449
  0.16007929 -0.16268692 -0.20219146 -0.18253636  0.04300045 -0.3688605
 -0.19044857 -0.12185212 -0.27844162  0.03433855 -0.2771063  -0.13832448
 -0.30395463  0.4399162  -0.08701513 -0.26

MAP 0.4849381590127316
--------------------------------------------------------------------------------------------
Cold_User_y57_ACF_RImg1_Edims200.npz
U shape (127, 64)
Y shape (57, 64)
A shape (57, 64)
E shape (200, 2048)
sum_alpha 0.014497360736674212
user id, testRS 3 [-0.01217793  0.05198475 -0.00345202  0.00811667 -0.12992283  0.27754344
  0.14316352 -0.09045396 -0.12683552  0.12850999 -0.05424909 -0.1004994
 -0.04962131 -0.09953126 -0.09037113  0.06961499  0.05556139 -0.14635431
 -0.06532282  0.01393086 -0.05284936 -0.1732862  -0.03395642 -0.13659459
  0.03917616  0.12765375  0.27489425 -0.01425755 -0.11629324] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [ 0.20224517  0.55293121  0.27299882 -0.22179245 -0.00957481 -0.2683904
 -0.20005879  0.54295811 -0.22580428 -0.08492835 -0.12591817 -0.31370982
  0.14022358 -0.16445447 -0.19553878 -0.17245853  0.00144227 -0.38913787
 -0.19343655 -0.11713851

all_sort [5, 26, 6, 9, 25, 15, 16, 1, 24, 19]
ideal [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
prec [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
all_sort [1, 7, 25, 2, 0, 12, 21, 16, 4, 9]
ideal [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
prec [1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
all_sort [2, 9, 25, 6, 0, 27, 15, 1, 28, 7]
ideal [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
prec [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
all_sort [18, 0, 13, 20, 1, 2, 22, 21, 10, 8]
ideal [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
prec [0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
all_sort [0, 1, 28, 3, 2, 21, 11, 13, 14, 6]
ideal [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
prec [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
all_sort [13, 28, 24, 8, 9, 4, 14, 15, 17, 5]
ideal [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
prec [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
all_sort [10, 0, 15, 2, 7, 6, 22, 24, 12, 17]
ideal [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
prec [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
all_sort [26, 21, 3, 11, 

ACF_NR

In [84]:
def testing(U, Y, A, E,Au, Ay, Aa, Av):

    test_amount = 25
    yt_test_amount = 29
    result=np.zeros((test_amount,57))
    RS=np.zeros((test_amount,57))
    #test_idx --> Test 的 index
    sum_alpha = 0
    for s in range(test_amount):
        #print(s,test_idx[s])

        yes=[]
        #sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        sample = train_t[test_idx[s]]
        #sample=result_yes_id[now]
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            alpha_a = np.dot(Au[test_idx[s]][sample[a]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[test_idx[s]][sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]][sample[a]],
                    np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]][sample[a]],np.dot(E,np.expand_dims(all_3374[sample[a]],0).T))
            """
            relu part ...
            """
            alpha[a]=np.sum((relu(alpha_a)))
            """
            tanh part ...
            """
            #alpha[a]=np.sum((np.tanh(alpha_a)))*r
            
            
        mul=np.zeros((1,64))
        #print('alpha--------',alpha)
        #print('add alpha------------',np.add(alpha,0.000000001))
        added_alpha = np.add(alpha,0.0000000001)
        norm_alpha = added_alpha/np.sum(added_alpha)
        #print('alpha-----------',alpha)
        #print('norm alpha--------------',norm_alpha)
        sum_alpha += np.sum(alpha)
        for i in range(len(sample)):
            mul+=norm_alpha[i]*A[sample[i]] #attention alpha*Ai part
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    
        
        
        for k in range(57):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)
            
    print('sum_alpha',sum_alpha)
    return RS

In [85]:
import os 
all_files = os.listdir(path)
new_all_files = []
for file in all_files:
    if 'y'+num_of_YouTuber in file and 'VBPR' not in file and 'ACF_NR' in file:
        new_all_files.append(file)
all_files = new_all_files
print(all_files)

['Cold_User_y57_ACF_NRImg0_Edims200.npz', 'Cold_User_y57_ACF_NRImg1_Edims200.npz']


In [86]:
par_files = ['Cold_User_y'+num_of_YouTuber+'_ACF_NR']
print(par_files)
for par_file in par_files:
    NDCG_List = []
    MAP_List = []
    DIM_List = []
    TOP1_Prec = []
    TOP3_Prec = []
    TOP5_Prec = []
    TOP1_F1 = []
    TOP3_F1 = []
    TOP5_F1 = []
    TOP1_Recall = []
    TOP3_Recall = []
    TOP5_Recall = []
    
    csv_path = './Result/csvResult/'
    for file in all_files:
        if par_file in file:
            print(file)
            par_data = np.load(path+file)
            U = par_data['U']
            Y = par_data['Y']
            A = par_data['A']
            E = par_data['E']
            #W1 = par_data['W1']
            Wu = par_data['Wu']
            Wy = par_data['Wy']
            Wa = par_data['Wa']
            Wv = par_data['Wv']
            RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv)
            target,testRS,sumtarget = getScoreMatrix(RS)
            prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
            prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
            prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
            pre_matrix,avg_ndcg = NDCG(target,testRS)
            Map_value = MAP(target,testRS)
            NDCG_List.append(avg_ndcg)
            MAP_List.append(Map_value)
            DIM_List.append(file.split('_')[3])
            TOP1_Prec.append(prec_1)
            TOP1_Recall.append(recall_1)
            TOP1_F1.append(f1_1)
            TOP3_Prec.append(prec_3)
            TOP3_Recall.append(recall_3)
            TOP3_F1.append(f1_3)
            TOP5_Prec.append(prec_5)
            TOP5_Recall.append(recall_5)
            TOP5_F1.append(f1_5)
            print('--------------------------------------------------------------------------------------------')
    #print(NDCG_List)
    #print(DIM_List)
    result_dict = {'Dims':DIM_List,'NDCG':NDCG_List,'MAP':MAP_List,
                   'TOP1 Precision':TOP1_Prec,'TOP1 Recall':TOP1_Recall,'TOP1 F1':TOP1_F1,
                  'TOP3 Precision':TOP3_Prec,'TOP3 Recall':TOP3_Recall,'TOP3 F1':TOP3_F1,
                  'TOP5 Precision':TOP5_Prec,'TOP5 Recall':TOP5_Recall,'TOP5 F1':TOP5_F1}
    df = pd.DataFrame(result_dict)
    df.to_csv(csv_path+par_file+'.csv',index=False)

['Cold_User_y57_ACF_NR']
Cold_User_y57_ACF_NRImg0_Edims200.npz
sum_alpha 0.02205023825808894
user id, testRS 3 [-0.05234857  0.08697921  0.1130548  -0.08153776 -0.10488853  0.27943983
 -0.02011668 -0.08036715 -0.07061537  0.13594098 -0.14586989 -0.06796038
 -0.02394079 -0.164344   -0.08305232  0.09304391  0.14669212 -0.20851348
 -0.07580313 -0.02156375 -0.0114803  -0.21213342 -0.01540553 -0.13360776
 -0.03215045  0.18680881  0.33553432  0.07537793 -0.13620912] [30, 47, 55, 48, 13, 26, 17, 11, 24, 10, 4, 8, 40, 43, 28, 44, 42, 0, 38, 54, 36, 46, 34, 18, 33, 6, 51, 21, 20]
user id, testRS 6 [ 0.12155154  0.43981213  0.07097623 -0.21766651  0.10011307 -0.12094349
 -0.17854384  0.28835368 -0.21138639  0.024018   -0.06699209 -0.24330636
  0.11270047 -0.06511431 -0.19063243 -0.17648825  0.09651209 -0.26663585
 -0.15343736 -0.04042669 -0.17255055 -0.01967145 -0.22881039 -0.168169
 -0.18109356  0.41591903 -0.07273235 -0.06122808 -0.18895677] [10, 51, 17, 14, 27, 22, 1, 26, 29, 12, 11, 19, 4, 2

MAP 0.41333520004820307
--------------------------------------------------------------------------------------------
